# Deep KNN

Este notebook tem como objetivo propor uma maneira alternativa de realizar classificação de imagens com deep learning, atravez da combinação de redes neurais profundas pretreinadas e da tecnica Knn. Esta combinação visa unir os beneficios da representação hierarquica de uma rede neural ja treinada com a necessidade de menos dados menos dados de um Knn. 

# Modelo proposto

### Arquitetura

O modelo aqui proposto consiste na utilização de uma rede pretreinada chamada [InceptionV3](https://keras.io/applications/#inceptionv3) sem sua ultima camada (predictions) e uma implementação do algoritimo Knn, desta forma a saida da rede neural deixa de ser a classificação original e passa a ser um vetor de 2048 posições que sera utilizado como entrada para o Knn.

### Fase de treino

Para realizar o treinamento o dataset de treino deve passar pela rede neural no sentido forward e o seu resultado deve ser passado ao Knn com a sua respectiva classe para treina-lo.

### Fase de predição

A realização da predição é bem similar a realização do treino, para ser classificada uma foto deve percorrer toda a rede neural no sentido forward e sua saida de deve ser passada ao Knn que ira finalmente realizar a classificação em si.

# Imports

Aqui estão contidas todas as bibliotecas importadas neste projeto

In [4]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

import keras.applications.inception_v3 as iv3

import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


# Coleta e tratamento de dados

## Sobre os dados

Neste trabalho iremos utilizar o dataset [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats) que pode ser obitido a partir de kaggle, este dataset contem 25,000 fotos de cães e gatos, este dataset foi criado fundamentalmente para que fosse possivel treinar algoritimos capazes de distuinguir de forma precisa um cachorro de um gato, e aqui nos o utilizaremos exatamente desta forma :)

## divisão dos dados

Os dados seram divididos em dois grupos, treino e teste, partindo do presuposto de que a rede pretreinada realizara um bom preprocessamento nos dados aqui adotaremos um holdout com proporções diferentes das tradicionais, utilizaremos aproximadamente 5%(50 instancias) dos dados para treino e 95%(800 instancias) para teste, sendo que cada grupo possui exatamente uma metade classificada como gato e a outra como cachorro.

### Dados de treino

In [5]:
datagen = ImageDataGenerator(rescale=1./255)

short_train = datagen.flow_from_directory('./short_train',
                                          target_size=(299, 299),
                                          batch_size=50,
                                          class_mode='categorical')

Found 50 images belonging to 2 classes.


In [6]:
X_train, y_train = next(short_train)

### Dados de teste

In [7]:
teste_images = datagen.flow_from_directory('./animals_test',
                                           target_size=(299, 299),
                                           batch_size=800,
                                           class_mode='categorical')

Found 800 images belonging to 2 classes.


In [8]:
X_test, y_test = next(teste_images)

# Montagem do modelo

## Corte na rede inception pretreinada

Aqui sera importado o modelo base citado anteriormente

In [9]:
base_model = iv3.InceptionV3(weights='imagenet')

O novo modelo tera todos os pesos de todas as camadas do modelo pretreinado com excessão da ultima camada que foi retirada

In [10]:
deep_featuring_model = Model(inputs=base_model.input, 
                             outputs=base_model.get_layer('avg_pool').output)

# Preprocessamento

Aqui faremos a primeira parte da faze de treinamento como descrita no item [Fase de treino](#Fase-de-treino) portanto passaremos todas as fotos do conjunto de treino pela rede neural que realizará o preprocessamento. Como um dos objetivos deste documento é comparar esta abordagem com outras aqui nos mediremos o tempo

### Preprocessamento dos dados de treino

In [11]:
%%time
features = deep_featuring_model.predict(X_train)

Wall time: 23.4 s


Para facilitar a leitura dos resultados susbstituiremos o vetor de tamanho 2 por uma string que indica se há um gato ou um cachorro na imagem

In [20]:
y_train = [ "dog" if str(label) == "[0. 1.]" else "cat" for label in y_train ]

### Preprocessamento dos dados de teste

In [23]:
%%time
features_test = deep_featuring_model.predict(X_test)

Wall time: 5min 42s


In [26]:
y_test = [ "dog" if str(label) == "[0. 1.]" else "cat" for label in y_test ]

# Treinamento do Knn com features preprocessadas

In [22]:
clf = KNeighborsClassifier(5)
clf.fit(features, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

# Fase de teste do modelo proposto

In [24]:
pred_y = clf.predict(features_test)

In [27]:
accuracy_score(y_test, pred_y)

0.9775

In [28]:
confusion_matrix(y_test, pred_y)

array([[387,  13],
       [  5, 395]], dtype=int64)